In [7]:
!pip install pyspark
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 2s (124 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
54 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping a

In [8]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('transactions_train.csv', header=True, inferSchema=True)
transactions_rdd = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Porcentaje de fraude

In [21]:
transactions_rdd.map(lambda x: x.target).distinct().collect()

[0, 1]

In [26]:
transactions_rdd.filter(lambda x: x.target == 1).count() * 100 / transactions_rdd.count()

3.502278074504093

Porcentaje de monto fraudulento

In [68]:
fraude = transactions_rdd.filter(lambda x: x.target == 1).map(lambda x: (x.product_category, x.ammount)).reduceByKey(lambda a,b: a + b)

In [69]:
legal = transactions_rdd.filter(lambda x: x.target == 0).map(lambda x: (x.product_category, x.ammount)).reduceByKey(lambda a,b: a + b)

In [72]:
producto_monto_f_l = fraude.fullOuterJoin(legal).collect()

In [74]:
def porcentaje(lista):
  promedios = []
  for producto in lista:
    promedios.append((producto[0], producto[1][0] * 100 / (producto[1][0] + producto[1][1])))
  return promedios

porcentaje(producto_monto_f_l)

[('H', 10.087869182508362),
 ('R', 5.134079848889628),
 ('S', 7.983558441966235),
 ('C', 12.916996422843251),
 ('W', 3.0548043859574134)]